In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import cv2
import torchvision.transforms.transforms as transforms

In [2]:
class FER2013(Dataset):
    """
    FER2013의 Custom Dataset.
    
    FER2013 데이터셋 탐색하기.ipynb 참고하여 작성하기
    """
    def __init__(self, path='./data', mode = 'train', transform = None):
        ## 여기에 코드 작성
        self.path = path + '/fer2013.csv'
        assert mode in ['train', 'val', 'test']
        self.mode = mode
        self.transform = transform
        self.data = pd.read_csv(self.path)
        
        if self.mode == 'train':
            self.data = self.data[self.data['Usage'] == 'Training']
        elif self.mode == 'val':
            self.data = self.data[self.data['Usage'] == 'PrivateTest']
        else:
            self.data = self.data[self.data['Usage'] == 'PublicTest']

    def __len__(self) -> int:        
        ## 여기에 코드 작성
        return self.data.index.size
    
    def __getitem__(self, index: int):
        ## 여기에 코드 작성
        item = self.data.iloc[index]
        
        emotion = item['emotion']
        pixels  = item['pixels']

        face = list(map(int, pixels.split(' ')))
        face = np.array(face).reshape(48,48).astype(np.uint8)
        
        # transform 적용
        if self.transform:
            # 학습 진행을 원활히 하기 위해 히스토그램 평활화를 적용
            face = cv2.equalizeHist(face)
            face = self.transform(face)

        return face, emotion


def create_train_dataloader(root='./data', batch_size=16):
    """
    train용 dataloader 함수
    
    FER2013 데이터셋 탐색하기.ipynb 참고하여 작성하기
    """
    ## 여기에 코드 작성
    transform = transforms.Compose([transforms.ToPILImage(), transforms.RandomHorizontalFlip(p=0.5), transforms.ToTensor()])
    dataset = FER2013(root, mode='train', transform=transform)
    dataloader = DataLoader(dataset, batch_size, shuffle=True)
    
    return dataloader

def create_val_dataloader(root='./data', batch_size=16):
    """
    validation용 dataloader 함수
    
    FER2013 데이터셋 탐색하기.ipynb 참고하여 작성하기
    """
    ## 여기에 코드 작성
    transform = transforms.Compose([transforms.ToPILImage(), transforms.RandomHorizontalFlip(p=0.5), transforms.ToTensor()])
    dataset = FER2013(root, mode='val', transform=transform)
    dataloader = DataLoader(dataset, batch_size, shuffle=False)
    
    return dataloader

def create_test_dataloader(root='./data', batch_size=16):
    """
    test용 dataloader 함수
    
    FER2013 데이터셋 탐색하기.ipynb 참고하여 작성하기
    """
    ## 여기에 코드 작성
    transform = transforms.Compose([transforms.ToPILImage(), transforms.RandomHorizontalFlip(p=0.5), transforms.ToTensor()])
    dataset = FER2013(root, mode='test', transform=transform)
    dataloader = DataLoader(dataset, batch_size, shuffle=False)   
    
    return dataloader 


    

In [ ]:
# dataloader_train = create_train_dataloader('/content/drive/MyDrive/NLP/data') 이런 식으로 불러 오면 됨
# 이름을 dataloader = .. 으로 하니까 오류 ..why?